In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
df = pd.read_csv("data/timeseries.csv", index_col="Date")
df.index = pd.to_datetime(df.index)
df = df.round(4)

In [34]:
train = df.iloc[:350]
test = df.iloc[350:]

In [35]:
X_train, y_train = train.drop(["EXCESS_RETURN_T+1"], axis=1).values, train["EXCESS_RETURN_T+1"].values
X_test, y_test = test.drop(["EXCESS_RETURN_T+1"], axis=1).values, test["EXCESS_RETURN_T+1"].values

In [36]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((350, 49), (350,), (165, 49), (165,))

In [46]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import ElasticNet


pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', ElasticNet())
])

param_grid = {
    'regressor__alpha': [i for i in np.arange(0.1,100,5)],  
    'scaler__feature_range': [(0, 1),(0, 5)],
    'regressor__l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
}

tscv = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(pipeline, param_grid, cv=tscv)

grid_search.fit(X_train, y_train)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('regressor', ElasticNet())]),
             param_grid={'regressor__alpha': [0.1, 5.1, 10.1, 15.1, 20.1, 25.1,
                                              30.1, 35.1, 40.1, 45.1, 50.1,
                                              55.1, 60.1, 65.1, 70.1, 75.1,
                                              80.1, 85.1, 90.1, 95.1],
                         'regressor__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6,
                                                 0.7, 0.8, 0.9, 1],
                         'scaler__feature_range': [(0, 1), (0, 5)]})

In [47]:
results = pd.DataFrame(grid_search.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,param_regressor__l1_ratio,param_scaler__feature_range,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002002,0.001416,0.002213,0.001488,0.1,0.1,"(0, 1)","{'regressor__alpha': 0.1, 'regressor__l1_ratio...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
1,0.003214,0.001270,0.000918,0.000955,0.1,0.1,"(0, 5)","{'regressor__alpha': 0.1, 'regressor__l1_ratio...",-0.019023,0.014301,-0.033416,-0.012713,0.019985,1
2,0.002203,0.001168,0.000498,0.000705,0.1,0.2,"(0, 1)","{'regressor__alpha': 0.1, 'regressor__l1_ratio...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
3,0.001684,0.002381,0.000450,0.000637,0.1,0.2,"(0, 5)","{'regressor__alpha': 0.1, 'regressor__l1_ratio...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
4,0.001387,0.000981,0.000000,0.000000,0.1,0.3,"(0, 1)","{'regressor__alpha': 0.1, 'regressor__l1_ratio...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.000187,0.000265,0.002003,0.002833,95.1,0.8,"(0, 5)","{'regressor__alpha': 95.1, 'regressor__l1_rati...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
396,0.002851,0.004031,0.000000,0.000000,95.1,0.9,"(0, 1)","{'regressor__alpha': 95.1, 'regressor__l1_rati...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
397,0.002077,0.001499,0.000257,0.000364,95.1,0.9,"(0, 5)","{'regressor__alpha': 95.1, 'regressor__l1_rati...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
398,0.001536,0.001799,0.000352,0.000497,95.1,1,"(0, 1)","{'regressor__alpha': 95.1, 'regressor__l1_rati...",-0.015129,-0.008891,-0.033056,-0.019025,0.010243,2
